<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/main/lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 6

## Agenda

1. Guest lectures
1. If there's time: Government/coding Q&A
1. Wrapping up assignments
1. Course evaluations

## [Guest lecture](https://docs.google.com/presentation/d/1rv-julgUaZFMOm1MlcP-hWWbxl5DHb3VALiZliNjJE8/edit)

[Ekaterina Levitskaya](https://www.linkedin.com/in/ekaterina-levitskaya-aa464426/)

> Ekaterina Levitskaya is an Associate Research Scientist at the [Coleridge Initiative](https://coleridgeinitiative.org/), New York University. She is a graduate of New York University’s Center for Urban Science and Progress (CUSP) and City University of New York (The Graduate Center) Computational Linguistics program. She utilizes computational approaches to the social science research, with special focus on text analysis and natural language processing. Her background is in computational linguistics, applied data science, and urban informatics. She is interested in applying computational skills for the projects with social impact and utilizing text as data in a variety of applications for the social science research.

## Guest lecture: [Building Ethical AI in the Federal Government](https://docs.google.com/presentation/d/1VWbrJxbOh8G6jL8R0iwUrM2LFuoGJz6cP2oxpQGFLOA/edit)

Shaudi Hosseini

## Me!

[![Screenshot of C2/Communicart](img/c2.png)](https://web.archive.org/web/20190124112324/https://cap.18f.gov/)

## [ATO Sprinting Team](https://18f.gsa.gov/2018/07/19/taking-the-ato-process-from-6-months-to-30-days/)

![Scatter chart with trendline showing decrease in ATO times](https://18f.gsa.gov/assets/blog/ato/ato-completion-date-graph.png)

## Agile budgeting

![Jellybeans](https://upload.wikimedia.org/wikipedia/commons/3/3e/Jellybeans.jpg)

## See also

[![Video still of man wearing suit at a microphone](https://img.youtube.com/vi/g-h6CtSwk30/maxresdefault.jpg)](https://www.youtube.com/watch?v=g-h6CtSwk30)

[Colleague explaining agile software development to the Michigan Senate](https://www.youtube.com/watch?v=g-h6CtSwk30)

## Questions!

- Working as a software developer
- Working in government
- Bread…?

## [Homework 6](https://colab.research.google.com/github/afeld/python-public-policy/blob/main/hw_5.ipynb#scrollTo=DRpVqnixIeFu)

- Giving feedback on proposals
- Encourage looking through them for ideas

## Late work

No assignment submissions will be accepted after one week from today, Homework 6 included.

## [Course evaluations](https://m.albert.nyu.edu/app/student/nyuCrseEval/crseEval/1214/6508/GP/10)

Please complete before signing off! The are:

- Totally anonymous
- A big help. Some things I took from the past:
    - Making assignments more rigorous
    - Students are hungry for more
    - People like the in-class exercises

## Thank you!

- alf9@nyu.edu
- [@aidanfeldman](https://twitter.com/aidanfeldman/) on Twitter
- Come to [Hacker Hours](https://hackerhours.org/)!